In [1]:
import random
import pandas as pd
import Covid19_model as CM
import numpy as np
import shutil
import sys
import os.path
import math
import csv
import lhsmdu
import time
from Pyomo_solve import solve_ode

# Set parameter

In [2]:
D = 1e2
n_site = 4
pop = np.array(pd.read_csv("population_"+str(n_site)+".csv",header=None))
pop = np.reshape(np.ones((6, 1)) * pop, (6, n_site))
init_state = np.array(pd.read_csv("state_"+str(n_site)+".csv",header=None))
density = np.array(pd.read_csv("density_"+str(n_site)+".csv",header=None))
dij = np.array(pd.read_csv("distance_"+str(n_site)+".csv",header=None)) / D
N = np.sum(init_state)
n_state = 7 * n_site + 1
n_action = 5 * n_site
density = np.reshape(density, (1, n_site))
T = 90
budget = 1e6
length = 40

In [3]:
deter_para = dict(N=N, P=pop, B=budget, T=T, D=dij, site=n_site, density=density,
                  gamma=0.69, alpha=0.6, v_max = 0.2/14,
                  cost_ti=0.0977, cost_ta=0.02, cost_v=0.07, 
                  cost_poc_0=0.000369, cost_poc_1=0.001057, 
                  pid=1.10/1000, psr=0.7/3, pid_plus=0.1221/1000, pir=1/8, prs=1/180)

in_deter_para = dict(beta2=[0.15204, 0.16287], beta1=[0.7602, 0.81435],
                     pei=[0.07143, 0.14286], per=[0.04000,0.05556])

in_deter_truth = dict(beta2=0.15747, beta1=0.78735, pei=0.10714, per=0.04545)

para_truth = {**deter_para, **in_deter_truth}

In [4]:
seed = 0
env = CM.Env_model(init_state, deter_para, in_deter_para, seed)
env.set_para_truth(para_truth)

In [5]:
tic = time.time()
random.seed(seed)
observation, state = init_state.copy(), init_state.copy()
state_buffer, new_buffer  = np.zeros([T, 6]), np.zeros([T, 4])
score, ni, ne = 0, 0, 0
done = False
index = [0]
start_time = 6
#n_select = 10
for t in range(T):
    p_set = env.para_sampling(100)
    para = p_set[index[0]]
    action_buffer = [] 
    reward_buffer = []
    s = np.array(state)
    ac = []
    if t > start_time:
        for a in range(n_site):
            try:
                aa = solve_ode(s[:,a], np.sum(s[1:-1,a])*budget/np.sum(s[1:-1,:]), para, length)
            except Exception:
                print("pyomo fail")
            ac.append(aa)
        action = np.array(ac).T
        if budget < 0: 
            action = np.zeros((5,n_site))
    else:
        action = np.zeros((5,n_site))
        
    
    next_state, obs, reward, cost, new_i, new_e, ck = env.online_state_estimate(state, observation, 
                                                                                action, p_set, 100, 1)
    #if not done and t > start_time: 
     #   env.space_update(ck, p_set, n_select)
    index = ck.argsort()[0][:1]
    score += reward
    budget -= cost
    ni += new_i
    ne += new_e

    new_buffer[t, 0], new_buffer[t, 1], new_buffer[t, 2], new_buffer[t, 3] = new_i, ni, new_e, ne
    print('\r', 'True State Time: {} | S: {:.1f}, E: {:.1f}, A: {:.1f}, I: {:.1f}, D: {:.1f}, R: {:.1f}, Budget: {:.2f}, score:{:.2f}'
          .format(t, np.sum(observation[0]), np.sum(observation[1]), np.sum(observation[2]), 
                  np.sum(observation[3]), np.sum(observation[4]), np.sum(observation[5]),
                  budget, score))
    print('\r', 'Observe Time: {} | S: {:.1f}, E: {:.1f}, A: {:.1f}, I: {:.1f}, D: {:.1f}, R: {:.1f}'
          .format(t, np.sum(state[0]), np.sum(state[1]), np.sum(state[2]), np.sum(state[3]),
                  np.sum(state[4]), np.sum(state[5]))) 
    print('\r', 'Time: {} | new_e:{:.1f}, cum_e:{:.1f}, new_i:{:.1f}, cum_i:{:.1f}'.format(t, new_e, ne, new_i, ni))
    state_buffer[t, :] = sum(observation.T)
    state = next_state.copy()
    observation = obs.copy()
print(time.time() - tic)

 True State Time: 0 | S: 12563972.0, E: 28.0, A: 0.0, I: 0.0, D: 0.0, R: 0.0, Budget: 1000000.00, score:-57.32
 Observe Time: 0 | S: 12563972.0, E: 28.0, A: 0.0, I: 0.0, D: 0.0, R: 0.0
 Time: 0 | new_e:13.2, cum_e:13.2, new_i:11.8, cum_i:11.8
 True State Time: 1 | S: 12563950.0, E: 37.0, A: 0.0, I: 11.8, D: 0.0, R: 1.3, Budget: 1000000.00, score:-138.59
 Observe Time: 1 | S: 12563949.0, E: 36.0, A: 0.0, I: 11.8, D: 0.0, R: 0.0
 Time: 1 | new_e:18.6, cum_e:31.8, new_i:16.3, cum_i:28.2
 True State Time: 2 | S: 12563919.0, E: 49.9, A: 0.0, I: 28.1, D: 0.0, R: 2.9, Budget: 1000000.00, score:-254.10
 Observe Time: 2 | S: 12563919.0, E: 50.0, A: 0.0, I: 28.1, D: 0.0, R: 0.0
 Time: 2 | new_e:26.2, cum_e:58.0, new_i:22.8, cum_i:51.0
 True State Time: 3 | S: 12563875.3, E: 68.5, A: 0.0, I: 50.9, D: 0.0, R: 5.2, Budget: 1000000.00, score:-418.56
 Observe Time: 3 | S: 12563877.0, E: 68.0, A: 0.0, I: 50.9, D: 0.0, R: 3.0
 Time: 3 | new_e:37.2, cum_e:95.2, new_i:32.1, cum_i:83.1
 True State Time: 4

 True State Time: 30 | S: 11834950.6, E: 29.0, A: 34.0, I: 130.5, D: 1.3, R: 728854.6, Budget: 609965.49, score:-2988.04
 Observe Time: 30 | S: 11835134.0, E: 0.0, A: 34.0, I: 130.5, D: 1.3, R: 728849.0
 Time: 30 | new_e:12.9, cum_e:671.7, new_i:15.3, cum_i:892.5
 True State Time: 31 | S: 11807482.9, E: 31.5, A: 34.8, I: 129.5, D: 1.3, R: 756320.0, Budget: 598910.69, score:-3049.37
 Observe Time: 31 | S: 11807687.0, E: 0.0, A: 34.8, I: 129.5, D: 1.3, R: 756314.0
 Time: 31 | new_e:13.9, cum_e:685.6, new_i:16.4, cum_i:908.9
 True State Time: 32 | S: 11783948.6, E: 34.9, A: 35.2, I: 129.7, D: 1.3, R: 779850.3, Budget: 586177.09, score:-3117.04
 Observe Time: 32 | S: 11784175.0, E: 0.0, A: 35.2, I: 129.7, D: 1.3, R: 779843.0
 Time: 32 | new_e:15.4, cum_e:701.0, new_i:17.8, cum_i:926.7
 True State Time: 33 | S: 11755311.2, E: 38.3, A: 36.5, I: 131.2, D: 1.3, R: 808481.5, Budget: 574068.98, score:-3191.10
 Observe Time: 33 | S: 11755562.0, E: 0.0, A: 36.5, I: 131.2, D: 1.3, R: 808474.0
 Time

 True State Time: 55 | S: 11275079.9, E: 5.1, A: 250.4, I: 483.2, D: 2.3, R: 1288179.0, Budget: 265967.43, score:-7259.08
 Observe Time: 55 | S: 11276654.0, E: 0.0, A: 250.4, I: 483.2, D: 2.3, R: 1288137.0
 Time: 55 | new_e:2.1, cum_e:1641.9, new_i:28.8, cum_i:2113.6
 True State Time: 56 | S: 11261680.3, E: 6.2, A: 212.5, I: 451.6, D: 2.4, R: 1301647.1, Budget: 257942.03, score:-7273.63
 Observe Time: 56 | S: 11263254.0, E: 0.0, A: 212.5, I: 451.6, D: 2.4, R: 1301607.0
 Time: 56 | new_e:2.6, cum_e:1644.5, new_i:25.2, cum_i:2138.7
    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Ipopt 3.14.6\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
 True State Time: 57 | S: 11245732.9, E: 7.4, A: 180.4, I: 420.2, D: 2.5, R: 1317656.6, Budget: 251793.17, score:-7451.61
 Observe Time: 57 | S: 11247307.0, E: 0.0, A: 180.4, I: 420.2, D: 2.5, R: 1317622.0
 Time: 57 | new_e:38.5, cum_e:1683.0, new_i:45.8, cum_i:2184.5
    m

 True State Time: 78 | S: 11136169.8, E: 22.1, A: 62.8, I: 142.7, D: 3.3, R: 1427599.3, Budget: 65804.10, score:-8197.39
 Observe Time: 78 | S: 11137911.0, E: 3.0, A: 62.8, I: 142.7, D: 3.3, R: 1427595.0
 Time: 78 | new_e:9.2, cum_e:1846.4, new_i:15.2, cum_i:2490.1
 True State Time: 79 | S: 11144057.4, E: 13.3, A: 67.7, I: 140.1, D: 3.3, R: 1419718.1, Budget: 64034.45, score:-8222.40
 Observe Time: 79 | S: 11145812.0, E: 0.0, A: 67.7, I: 140.1, D: 3.3, R: 1419713.0
 Time: 79 | new_e:5.5, cum_e:1852.0, new_i:12.4, cum_i:2502.5
 True State Time: 80 | S: 11147852.3, E: 16.3, A: 57.9, I: 134.9, D: 3.3, R: 1415935.2, Budget: 60853.37, score:-8252.67
 Observe Time: 80 | S: 11149616.0, E: 0.0, A: 57.9, I: 134.9, D: 3.3, R: 1415931.0
 Time: 80 | new_e:6.8, cum_e:1858.7, new_i:12.5, cum_i:2515.0
 True State Time: 81 | S: 11146572.7, E: 20.1, A: 49.5, I: 130.5, D: 3.3, R: 1417223.8, Budget: 57483.32, score:-8289.82
 Observe Time: 81 | S: 11148346.0, E: 0.0, A: 49.5, I: 130.5, D: 3.3, R: 1417220.

In [2]:
2363.687 /(T - start_time + 1)


27.808082352941174